In [ ]:
-- All Blocking Queries  
-- To Terminate a locking Session type:   kill session_id
SELECT qs.Session_ID
	,Blocking_Session_ID
	,qs.STATUS
	,Wait_Type
	,Wait_Time
	,Wait_Resource
	,SUBSTRING(st.TEXT, (qs.statement_start_offset / 2) + 1, (
			(
				CASE qs.statement_end_offset
					WHEN - 1
						THEN DATALENGTH(st.TEXT)
					ELSE qs.statement_end_offset
					END - qs.statement_start_offset
				) / 2
			) + 1) AS statement_text
	,GetDate() SnapshotDateTime
	,
	--Open_Transaction_Count,
	ss.PROGRAM_NAME
	,ss.HOST_NAME
	,ss.Login_Name
FROM sys.dm_exec_requests AS qs
INNER JOIN sys.dm_exec_sessions ss ON qs.session_id = ss.session_id
CROSS APPLY sys.dm_exec_sql_text(qs.sql_handle) AS st
WHERE Wait_Time > 0
ORDER BY Wait_Time DESC

In [ ]:
-- Find and Terminate blocking query.
--------------------------------------------------
-- Open in another query window and run...
--------------------------------------------------
SELECT * FROM Flavors_DW_Staging.JDE.ITEM_LEDGER_STAGING WITH (TABLOCKX) -- Creates lock against table while reading it.

--------------------------------------------------
-- Open in another query window and run...
--------------------------------------------------
-- Kills lock reading my table.
DECLARE @BadSession INT
DECLARE @MSSQL AS VARCHAR(100)
DECLARE @i int = 0 --loop number

    WHILE @i < 10 -- Number of retries
	BEGIN
	SET @i = @i + 1
	SET @BadSession = ISNULL((SELECT TOP 1 qs.Session_ID 
					FROM sys.dm_exec_requests AS qs 
					INNER JOIN sys.dm_exec_sessions ss ON qs.session_id = ss.session_id 
					CROSS APPLY sys.dm_exec_sql_text(qs.sql_handle) AS st 
					WHERE Wait_Time > 0 AND st.TEXT LIKE '%ITEM_LEDGER_STAGING%'),0) -- Change to object/table name
	--PRINT @BadSession
	IF 0 > @BadSession 
	BEGIN
	SET @MSSQL = 'kill ' + CONVERT(varchar(18),@BadSession)
    EXEC(@MSSQL)
	END 

	WAITFOR DELAY '00:00:01' -- Wait between retries
    END
---- Free to do anything to the table no locks or locks been terminated.

In [ ]:
-- Current Sessions to DB
SELECT conn.session_id, sson.host_name, sson.login_name, 
 sqltxt.text, sson.login_time,  sson.status, DB_NAME(sson.database_id) AS DBName
FROM sys.dm_exec_connections conn
INNER JOIN sys.dm_exec_sessions sson 
ON conn.session_id = sson.session_id
CROSS APPLY sys.dm_exec_sql_text(most_recent_sql_handle) AS sqltxt
ORDER BY conn.session_id

In [ ]:
-- Server Query History
SELECT TOP 50 * FROM(SELECT COALESCE(OBJECT_NAME(s2.objectid),'Ad-Hoc') AS ProcName,
  execution_count,s2.objectid,
    (SELECT TOP 1 SUBSTRING(s2.TEXT,statement_start_offset / 2+1 ,
      ( (CASE WHEN statement_end_offset = -1
  THEN (LEN(CONVERT(NVARCHAR(MAX),s2.TEXT)) * 2)
ELSE statement_end_offset END)- statement_start_offset) / 2+1)) AS sql_statement,
       last_execution_time
FROM sys.dm_exec_query_stats AS s1
CROSS APPLY sys.dm_exec_sql_text(sql_handle) AS s2 ) x
WHERE sql_statement NOT like 'SELECT TOP 50 * FROM(SELECT %'
--and OBJECTPROPERTYEX(x.objectid,'IsProcedure') = 1
ORDER BY last_execution_time DESC

In [ ]:
-- Query All users without limits in Active Directory once ADSI is setup as linked server (Change LDAP String to company)
DECLARE @SYMBOLS AS VARCHAR(128); DECLARE @query VARCHAR(MAX); DECLARE @ADfields VARCHAR(MAX);
SET @SYMBOLS = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'; -- AD Search character prefixes used to partition search. (1000 page limit). Below, AD fields to retrieve:
SET @ADfields = 'cn, sAMAccountName, accountExpires, pwdLastSet, userAccountControl, ADsPath, lockouttime, manager, mail, createTimeStamp, employeeID, lastLogon, co, l';
SET @query = 'SELECT * INTO ##tmpActiveDirectory FROM (';
 
-- Get each character and for each character construct and AD query:
WITH nmTbl AS (SELECT TOP (LEN(@SYMBOLS)) Idx = ROW_NUMBER() OVER (ORDER BY [object_id]) FROM sys.all_objects ORDER BY Idx)
SELECT @query = @query + 'SELECT * FROM OPENQUERY(ADSI, ''SELECT ' + @ADfields + ' FROM ''''LDAP://DC=ABC,DC=XYZ''''   
WHERE objectCategory=''''Person'''' AND (cn = ''''' + SUBSTRING(@SYMBOLS, nmTbl.Idx, 1) + '*'''') AND (objectClass = ''''user'''' OR objectClass = ''''contact'''')'')
UNION
' FROM nmTbl;
 
-- Finish generating query string:
SELECT @query = LEFT(@query, LEN(@query) - CHARINDEX(REVERSE('UNION'), REVERSE(@query)) - 4) + ') AS qry'
 
-- Remove temp table if existing before running:
IF OBJECT_ID('tempdb.dbo.##tmpActiveDirectory', 'U') IS NOT NULL
    DROP TABLE ##tmpActiveDirectory; 
EXECUTE(@query)
 
SELECT * FROM ##tmpActiveDirectory

In [ ]:
--Job Information & Step Information
SELECT a.name
	  ,a.enabled
      ,js.step_id
      ,js.step_name
      ,CONVERT(DATETIME,(
	  	CASE
		WHEN LEN(js.last_run_time) = 5 THEN 
		SUBSTRING(CONVERT(VARCHAR(8),js.last_run_date),5,2) + '/'+ RIGHT(js.last_run_date,2) + '/'+ LEFT(js.last_run_date,4)  + ' '+LEFT(LEFT(js.last_run_time,1) +':'+RIGHT(js.last_run_time,4),4)+'.'+RIGHT(js.last_run_time,2)
		WHEN LEN(js.last_run_time) = 6 THEN
		SUBSTRING(CONVERT(VARCHAR(8),js.last_run_date),5,2) + '/'+ RIGHT(js.last_run_date,2) + '/'+ LEFT(js.last_run_date,4)  + ' '+LEFT(LEFT(js.last_run_time,2) +':'+RIGHT(js.last_run_time,4),5)+'.'+RIGHT(js.last_run_time,2)
		END )) AS LastRan
	  ,CONVERT(varchar, js.last_run_duration / 86400 ) + ':' + CONVERT(varchar, DATEADD(ms, ( js.last_run_duration % 86400 ) * 1000, 0), 114) AS JobDurFormat
	  ,js.last_run_duration
	  ,js.database_name
      ,js.subsystem
      ,js.command
FROM  msdb.dbo.sysjobs  a 
LEFT JOIN msdb.dbo.sysjobsteps js ON js.job_id=a.job_id 
LEFT JOIN msdb.dbo.sysssispackages b ON a.name=b.name
--WHERE a.enabled = 1 AND js.subsystem = 'SSIS'

In [ ]:
-- Job Start Information & Schedule Information.
SELECT JobScheduleTime AS JobScheduleTime_sort
,CASE WHEN ISDATE(JobScheduleTime)=1 THEN FORMAT(CONVERT(DATETIME,JobScheduleTime,7),'hh\:mm tt') ELSE JobScheduleTime END JobScheduleTime
,JobName,	ScheduleName,Recurrence,Step,StepName
,NextRunDate, s1.JobDurFormat
FROM (
		SELECT DISTINCT j.name AS 'JobName', category_id
				 ----[description],
				 --s.step_id as 'Step',
				 --s.step_name as 'StepName',
		   -- --[schedule_uid] AS [ScheduleID]
			,sch.[name] AS [ScheduleName]
			--, CASE j.[enabled]
			--    WHEN 1 THEN 'Yes'
			--    WHEN 0 THEN 'No'
			--  END AS [IsEnabled]
			--, CASE 
			--    WHEN [freq_type] = 64 THEN 'Start automatically when SQL Server Agent starts'
			--    WHEN [freq_type] = 128 THEN 'Start whenever the CPUs become idle'
			--    WHEN [freq_type] IN (4,8,16,32) THEN 'Recurring'
			--    WHEN [freq_type] = 1 THEN 'One Time'
			--  END [ScheduleType]
			--, CASE [freq_type]
			--    WHEN 1 THEN 'One Time'
			--    WHEN 4 THEN 'Daily'
			--    WHEN 8 THEN 'Weekly'
			--    WHEN 16 THEN 'Monthly'
			--    WHEN 32 THEN 'Monthly - Relative to Frequency Interval'
			--    WHEN 64 THEN 'Start automatically when SQL Server Agent starts'
			--    WHEN 128 THEN 'Start whenever the CPUs become idle'
			--  END [Occurrence]
			, CASE [freq_type]
				WHEN 4 THEN CAST([freq_interval] AS VARCHAR(3)) + ' day(s)'
				WHEN 8 THEN   CASE WHEN [freq_interval] & 1 = 1 THEN 'Sun' ELSE '' END
							+ CASE WHEN [freq_interval] & 2 = 2 THEN ', Mon' ELSE '' END
							+ CASE WHEN [freq_interval] & 4 = 4 THEN ', Tue' ELSE '' END
							+ CASE WHEN [freq_interval] & 8 = 8 THEN ', Wed' ELSE '' END
							+ CASE WHEN [freq_interval] & 16 = 16 THEN ', Thu' ELSE '' END
							+ CASE WHEN [freq_interval] & 32 = 32 THEN ', Fri' ELSE '' END
							+ CASE WHEN [freq_interval] & 64 = 64 THEN ', Sat' ELSE '' END
				WHEN 16 THEN  CAST([freq_interval] AS VARCHAR(3)) 
							 + ' of every '
							 + CAST([freq_recurrence_factor] AS VARCHAR(3)) + ' month(s)'
				WHEN 32 THEN 
							 + CASE [freq_relative_interval]
								WHEN 1 THEN 'First'
								WHEN 2 THEN 'Second'
								WHEN 4 THEN 'Third'
								WHEN 8 THEN 'Fourth'
								WHEN 16 THEN 'Last'
							   END
							 + ' ' 
							 + CASE [freq_interval]
								WHEN 1 THEN 'Sun'
								WHEN 2 THEN 'Mon'
								WHEN 3 THEN 'Tue'
								WHEN 4 THEN 'Wed'
								WHEN 5 THEN 'Thu'
								WHEN 6 THEN 'Fri'
								WHEN 7 THEN 'Sat'
								WHEN 8 THEN 'Day'
								WHEN 9 THEN 'Weekday'
								WHEN 10 THEN 'Weekend day'
							   END
							 +  CAST([freq_recurrence_factor] AS VARCHAR(3)) 
							 + ' month(s)'
			  END AS [Recurrence]
			, CASE [freq_subday_type]
				WHEN 1 THEN STUFF(
						 STUFF(RIGHT('000000' + CAST([active_start_time] AS VARCHAR(6)), 6)
										, 3, 0, ':')
									, 6, 0, ':')
				WHEN 2 THEN 'Occurs every ' 
							+ CAST([freq_subday_interval] AS VARCHAR(3)) + ' Second(s) between ' 
							+ STUFF(
						   STUFF(RIGHT('000000' + CAST([active_start_time] AS VARCHAR(6)), 6)
										, 3, 0, ':')
									, 6, 0, ':')
							+ ' & ' 
							+ STUFF(
							STUFF(RIGHT('000000' + CAST([active_end_time] AS VARCHAR(6)), 6)
										, 3, 0, ':')
									, 6, 0, ':')
				WHEN 4 THEN 'Occurs every ' 
							+ CAST([freq_subday_interval] AS VARCHAR(3)) + ' Minute(s) between ' 
							+ STUFF(
						   STUFF(RIGHT('000000' + CAST([active_start_time] AS VARCHAR(6)), 6)
										, 3, 0, ':')
									, 6, 0, ':')
							+ ' & ' 
							+ STUFF(
							STUFF(RIGHT('000000' + CAST([active_end_time] AS VARCHAR(6)), 6)
										, 3, 0, ':')
									, 6, 0, ':')
				WHEN 8 THEN 'Occurs every ' 
							+ CAST([freq_subday_interval] AS VARCHAR(3)) + ' Hour(s) between ' 
							+ STUFF(
							STUFF(RIGHT('000000' + CAST([active_start_time] AS VARCHAR(6)), 6)
										, 3, 0, ':')
									, 6, 0, ':')
							+ ' & ' 
							+ STUFF(
							STUFF(RIGHT('000000' + CAST([active_end_time] AS VARCHAR(6)), 6)
										, 3, 0, ':')
									, 6, 0, ':')
			  END [JobScheduleTime]
			--, STUFF(
			--        STUFF(CAST([active_start_date] AS VARCHAR(8)), 5, 0, '-')
			--            , 8, 0, '-') AS [ScheduleUsageStartDate]
	     ,s.step_id AS 'Step'
		,s.step_name AS 'StepName'
		,NextRunDate=CASE WHEN ISDATE(CONVERT(VARCHAR(12),next_run_date))=1 THEN CONVERT(DATE,CONVERT(VARCHAR(12),next_run_date)) ELSE '' END
		  ,CONVERT(varchar, last_run_duration / 86400 ) + ':' + CONVERT(varchar, DATEADD(ms, ( last_run_duration % 86400 ) * 1000, 0), 114) AS JobDurFormat
		FROM msdb.dbo.sysjobs j
		JOIN msdb.dbo.sysjobsteps s  ON j.job_id = s.job_id
		JOIN msdb.dbo.sysjobschedules jsch ON j.job_id = jsch.job_id
		JOIN [msdb].[dbo].[sysschedules] sch ON jsch.schedule_id=sch.schedule_id
		WHERE j.enabled = 1 AND  sch.[name]<>'Schedule_1'
	) s1
WHERE NextRunDate>=CONVERT(DATE,GETDATE())
AND JobName NOT LIKE ('%Subplan%') 
AND JobName NOT LIKE ('SSIS%')
AND JobName NOT LIKE ('MDS%') AND JobName NOT LIKE ('Sys%')
--ORDER BY [JobScheduleTime],jobname,[ScheduleName],step

In [ ]:
-- Job Scheduled error outcome
SELECT 
CONVERT(DATETIME,(
	  	CASE
		WHEN LEN(js.last_run_time) = 5 THEN 
		SUBSTRING(CONVERT(VARCHAR(8),js.last_run_date),5,2) + '/'+ RIGHT(js.last_run_date,2) + '/'+ LEFT(js.last_run_date,4)  + ' '+LEFT(LEFT(js.last_run_time,1) +':'+RIGHT(js.last_run_time,4),4)+'.'+RIGHT(js.last_run_time,2)
		WHEN LEN(js.last_run_time) = 6 THEN
		SUBSTRING(CONVERT(VARCHAR(8),js.last_run_date),5,2) + '/'+ RIGHT(js.last_run_date,2) + '/'+ LEFT(js.last_run_date,4)  + ' '+LEFT(LEFT(js.last_run_time,2) +':'+RIGHT(js.last_run_time,4),5)+'.'+RIGHT(js.last_run_time,2)
		END )) AS LastRan,
A.Name AS JobName, js.step_name AS StepName, js.step_id AS StepNbr, js.database_name AS UseDB,  js.subsystem AS CommandType, js.command,
C.message AS FailedOutcome,CONVERT(varchar, js.last_run_duration / 86400 ) + ':' + CONVERT(varchar, DATEADD(ms, ( js.last_run_duration % 86400 ) * 1000, 0), 114) AS JobDur, d.next_run_date
, CAST(STUFF(STUFF(CONCAT(REPLICATE('0', 6 - LEN(d.next_run_time)),d.next_run_time),3,0,':'),6,0,':') AS TIME(0)) AS NextRunTime
FROM  msdb.dbo.sysjobs  a 
LEFT JOIN msdb.dbo.sysjobsteps js ON js.job_id=a.job_id 
LEFT JOIN msdb.dbo.sysssispackages b ON a.name=b.name
INNER JOIN (SELECT * FROM msdb..sysjobhistory WHERE message LIKE '%failed%' ) AS c ON js.job_id = c.job_id
INNER JOIN( SELECT DISTINCT A.job_id, A.schedule_id, A.next_run_date, A.next_run_time, B.enabled 
FROM msdb.dbo.sysjobschedules AS A
INNER join msdb.dbo.sysschedules AS B
ON A.schedule_id = B.schedule_id
WHERE B.enabled = 1 
) AS d
ON A.job_id = D.job_id
WHERE a.enabled = 1 AND a.category_id = 0 AND js.last_run_date = c.run_date AND js.last_run_time = c.run_time 

In [ ]:
-- DB Last Used
SELECT @@SERVERNAME AS Server ,DB_NAME([database_id]) AS DatabaseName,
       MAX(last_user_seek) AS LastUserSeek, MAX(last_user_scan) AS LastUserScan,MAX([last_user_update]) AS LastUserUpdate,
	   (SELECT [sqlserver_start_time] FROM sys.dm_os_sys_info) AS LastReboot, DATEDIFF(d, (SELECT [sqlserver_start_time] FROM sys.dm_os_sys_info) , GETDATE()) DaySinceReboot
FROM sys.dm_db_index_usage_stats
WHERE DB_NAME([database_id]) IN (SELECT [name] FROM sys.databases WHERE [database_id] > 4AND [state_desc] = 'online')
GROUP BY [database_id]
ORDER BY LastUserSeek, LastUserScan, LastUserUpdate

-- Option 2
SELECT DatabaseName, MAX(LastAccessDate) DatabaseLastAccessedOn FROM (
 SELECT DB_NAME(database_id) DatabaseName , last_user_seek, last_user_scan, last_user_lookup, last_user_update
 FROM sys.dm_db_index_usage_stats) AS Pvt
UNPIVOT
 ( LastAccessDate FOR last_user_access IN (last_user_seek, last_user_scan , last_user_lookup , last_user_update)
 ) AS Unpvt
GROUP BY DatabaseName
HAVING DatabaseName NOT IN ('master', 'tempdb', 'model', 'msdb')

In [ ]:
-- Tables Last Used
SELECT DB_NAME(ius.[database_id]) AS [Database],
OBJECT_NAME(ius.[object_id]) AS [TableName],
MAX(ius.[last_user_lookup]) AS [last_user_lookup],
MAX(ius.[last_user_scan]) AS [last_user_scan],
MAX(ius.[last_user_seek]) AS [last_user_seek] FROM sys.dm_db_index_usage_stats AS ius
WHERE ius.[database_id] = DB_ID()
GROUP BY ius.[database_id], ius.[object_id]
ORDER BY [last_user_scan] ASC;

In [ ]:
-- Table Use audit.
DECLARE @MSSQL AS VARCHAR(8000);
DECLARE @DBName AS VARCHAR(50);

DECLARE GetName CURSOR FOR
SELECT name  FROM sys.databases
WHERE name IN ('DatabaseA','DatbaseB'); -- My List of Databases to Adit

OPEN GetName
FETCH NEXT FROM GetName INTO  @DBName 
WHILE @@FETCH_STATUS = 0 
BEGIN
--PRINT  @DBName;
SET @MSSQL = ' ---------------
DELETE FROM DWDiagnostics.dbo.DIM_TableUsage WHERE DatabaseName = '''+@DBName+''' AND ReportedDate = CONVERT(DATE,GETDATE(),101);
USE ' +@DBName+ '; 
INSERT INTO  DWDiagnostics.dbo.DIM_TableUsage ([ReportedDate],[DatabaseName] ,[SchemaName],[TableName],[TablePath],[RowCount],[MegaByteSize],[CreatedOn],[LastModifedOn],[DataChangeDate],[DataAgeDays],[LastDataRead],[LastUpdated],[LastExecution]) 
SELECT ReportedDate, DatabaseName, SchemaName,A.TableName,A.TablePath,[RowCount],A.MegaByteSize,A.CreatedOn,A.LastModifedOn,
	CASE
        WHEN ISNULL(A.LastDataRead,A.LastModifedOn) >=  ISNULL(A.LastUpdated,A.LastModifedOn) AND  ISNULL(A.LastDataRead,A.LastModifedOn) >= ISNULL(A.LastExecution,A.LastModifedOn) THEN  ISNULL(A.LastDataRead,A.LastModifedOn)
        WHEN ISNULL(A.LastUpdated ,A.LastModifedOn)>=  ISNULL(A.LastDataRead,A.LastModifedOn) AND  ISNULL(A.LastUpdated ,A.LastModifedOn)>=  ISNULL(A.LastExecution,A.LastModifedOn) THEN  ISNULL(A.LastUpdated,A.LastModifedOn)
        WHEN ISNULL(A.LastExecution,A.LastModifedOn)>=  ISNULL(A.LastDataRead,A.LastModifedOn) AND  ISNULL(A.LastExecution,A.LastModifedOn) >=  ISNULL(A.LastUpdated,A.LastModifedOn) THEN  ISNULL(A.LastExecution,A.LastModifedOn)
        ELSE  A.LastModifedOn
    END AS DataChangeDate,
	DATEDIFF(DAY,(	CASE
        WHEN ISNULL(A.LastDataRead,A.LastModifedOn) >=  ISNULL(A.LastUpdated,A.LastModifedOn) AND  ISNULL(A.LastDataRead,A.LastModifedOn) >= ISNULL(A.LastExecution,A.LastModifedOn) THEN  ISNULL(A.LastDataRead,A.LastModifedOn)
        WHEN ISNULL(A.LastUpdated ,A.LastModifedOn)>=  ISNULL(A.LastDataRead,A.LastModifedOn) AND  ISNULL(A.LastUpdated ,A.LastModifedOn)>=  ISNULL(A.LastExecution,A.LastModifedOn) THEN  ISNULL(A.LastUpdated,A.LastModifedOn)
        WHEN ISNULL(A.LastExecution,A.LastModifedOn)>=  ISNULL(A.LastDataRead,A.LastModifedOn) AND  ISNULL(A.LastExecution,A.LastModifedOn) >=  ISNULL(A.LastUpdated,A.LastModifedOn) THEN  ISNULL(A.LastExecution,A.LastModifedOn)
        ELSE  A.LastModifedOn END )
		,GETDATE()) AS DataAgeDays,
      A.LastDataRead,A.LastUpdated,A.LastExecution

FROM (
SELECT
	CONVERT(date,GETDATE(),101) AS ReportedDate,
	DB_NAME() AS DatabaseName,
	a3.name AS SchemaName,
	a2.name AS TableName,
	CONCAT(''['',DB_NAME(),''].['',a3.name,''].['',a2.name ,'']'') AS TablePath,
	a1.rows AS [RowCount],
	CONVERT(NUMERIC(18,1),CONVERT(NUMERIC(18,1),(a1.data * 8))/1024) AS MegaByteSize,
	CONVERT(DATE,a2.create_date) AS CreatedOn,
	CONVERT(DATE,a2.modify_date) AS LastModifedOn,
	CASE 
	WHEN ISNULL(A5.LastLookup,''1900-01-01 00:00:00.000'') = ''1900-01-01 00:00:00.000''
	AND  ISNULL(A5.LastScan,''1900-01-01 00:00:00.000'') = ''1900-01-01 00:00:00.000''
	AND  ISNULL(A5.LastSeek,''1900-01-01 00:00:00.000'') = ''1900-01-01 00:00:00.000''
	THEN NULL
	ELSE a5.LastScan
	END AS LastDataRead,
	A6.LastUpdated,
	(SELECT MAX(last_execution_time)  AS LastExecution
	FROM(SELECT COALESCE(OBJECT_NAME(s2.objectid),''Ad-Hoc'') AS ProcName,
	(SELECT TOP 1 SUBSTRING(s2.TEXT,statement_start_offset / 2+1 , ( (CASE WHEN statement_end_offset = -1 THEN (LEN(CONVERT(NVARCHAR(MAX),s2.TEXT)) * 2) ELSE statement_end_offset END)- statement_start_offset) / 2+1)) AS sql_statement
	,last_execution_time 
	FROM sys.dm_exec_query_stats AS s1
	CROSS APPLY sys.dm_exec_sql_text(sql_handle) AS s2 ) x
	WHERE sql_statement LIKE ''%''+a2.name +''%'' AND sql_statement LIKE ''%''+DB_NAME()+''%'' AND x.sql_statement NOT LIKE ''%sys.dm_exec_query_stats%''
	) AS LastExecution
FROM (
	SELECT ps.object_id,
		SUM (CASE WHEN (ps.index_id < 2) THEN row_count ELSE 0 END) AS [rows],
		SUM (ps.reserved_page_count) AS reserved,
		SUM (CASE WHEN (ps.index_id < 2) THEN (ps.in_row_data_page_count + ps.lob_used_page_count + ps.row_overflow_used_page_count) ELSE (ps.lob_used_page_count + ps.row_overflow_used_page_count) END) AS data,
		SUM (ps.used_page_count) AS used
	FROM sys.dm_db_partition_stats ps
        WHERE ps.object_id NOT IN (SELECT object_id FROM sys.tables WHERE is_memory_optimized = 1)
	GROUP BY ps.object_id) AS a1
LEFT OUTER JOIN (
     SELECT it.parent_id, SUM(ps.reserved_page_count) AS reserved, SUM(ps.used_page_count) AS used
	 FROM sys.dm_db_partition_stats ps
	 INNER JOIN sys.internal_tables it ON (it.object_id = ps.object_id)
	 WHERE it.internal_type IN (202,204)
	 GROUP BY it.parent_id
	 ) AS a4 
	 ON (a4.parent_id = a1.object_id)
INNER JOIN sys.all_objects a2  ON ( a1.object_id = a2.object_id ) 
INNER JOIN sys.schemas a3 ON (a2.schema_id = a3.schema_id)
LEFT JOIN (
SELECT ISNULL(DB_NAME(ISNULL(ius.[database_id],'''')),'''') AS [Database],
 ISNULL(OBJECT_NAME(ISNULL(ius.[object_id],''''), ISNULL(ius.[database_id],'''')),'''') AS [TableName],
 OBJECT_SCHEMA_NAME(ius.[object_id],ius.[database_id]) AS sch,
 MAX(ISNULL(ius.[last_user_lookup],'''')) AS [LastLookup],
 MAX(ISNULL(ius.[last_user_scan],'''') )AS [LastScan],
 MAX(ISNULL(ius.[last_user_seek],'''')) AS [LastSeek] 
FROM sys.dm_db_index_usage_stats AS ius
GROUP BY ius.[database_id], ius.[object_id]
) AS a5
ON a5.[Database] = DB_NAME() AND a5.sch = a3.name AND  a2.name = a5.TableName 
LEFT JOIN (
		 SELECT TABLENAME,sch, LASTUPDATED FROM 
        (
            SELECT  SCHEMA_NAME(B.schema_id) AS sch, B.NAME AS ''TABLENAME'', MAX(STATS_DATE (ID,INDID)) AS LastUpdated,
                    CONVERT(VARCHAR, MAX(STATS_DATE (ID,INDID)), 103) as Date
            FROM    SYS.SYSINDEXES AS A
                    INNER JOIN SYS.OBJECTS AS B ON A.ID = B.OBJECT_ID
            WHERE   B.TYPE = ''U''  AND STATS_DATE (ID,INDID) IS NOT NULL 
            GROUP BY B.NAME, B.schema_id
        ) AS A
) AS a6
ON A3.name = A6.sch AND a2.name = a6.TABLENAME
WHERE a2.type NOT IN (''S'',''IT'')
) AS A; ';

--PRINT @MSSQL;
EXEC (@MSSQL);
FETCH NEXT FROM GetName INTO @DBName 
END
CLOSE GetName
DEALLOCATE GetName

In [ ]:
-- Index % rebuild
DECLARE @DatabaseName VARCHAR(50)
DECLARE @FragPercent NVARCHAR(3)
SET @DatabaseName = 'Master'
SET @FragPercent = '30'
------------
DECLARE @i INT=0
DECLARE @IndexCount INT
DECLARE @IndexFregQuery NVARCHAR(MAX)
DECLARE @IndexRebuildQuery NVARCHAR(MAX)
DECLARE @IndexName VARCHAR(500)
DECLARE @TableName VARCHAR(500)
DECLARE @SchemaName VARCHAR(500)

DROP TABLE IF EXISTS #Fregmentedindex;
CREATE TABLE #Fregmentedindex(Index_name VARCHAR(MAX),table_name VARCHAR(MAX),schema_name VARCHAR(MAX))
SET @IndexFregQuery='SELECT i.[name],o.name,sch.name
	FROM   [' + @DatabaseName + '].sys.dm_db_index_physical_stats (DB_ID('''+ @DatabaseName +'''), NULL, NULL, NULL, NULL) AS s
	INNER JOIN [' + @DatabaseName + '].sys.indexes AS i ON s.object_id = i.object_id AND s.index_id = i.index_id
	INNER JOIN [' + @DatabaseName + '].sys.objects AS o ON i.object_id = o.object_id
	INNER JOIN [' + @DatabaseName + '].sys.schemas AS sch ON o.schema_id=sch.schema_id
	WHERE (s.avg_fragmentation_in_percent > '+@FragPercent+' ) and i.name is not null'
INSERT INTO #Fregmentedindex(Index_name,table_name,schema_name) EXEC sp_executesql @IndexFregQuery
SET @IndexCount=(SELECT COUNT(1) FROM #Fregmentedindex)
WHILE (@IndexCount>@i)
BEGIN 
(SELECT TOP 1 @TableName=table_name, @IndexName=Index_name,@SchemaName= schema_name FROM #Fregmentedindex)
SET @IndexRebuildQuery ='Alter index [' + @IndexName +'] on ['+@DatabaseName +'].['+@SchemaName+'].[' + @TableName +'] rebuild'
PRINT @IndexRebuildQuery 
EXEC sp_executesql @IndexRebuildQuery 
SET @i=@i+1
DELETE FROM #Fregmentedindex WHERE Index_name=@IndexName AND table_name=@TableName
END

In [ ]:
-- Check for Schema usage in Use by database 
SELECT s.name AS schema_name, u.name AS schema_owner,ISNULL(c.NoObjects,0) AS NbrOfObjects
FROM sys.schemas s 
INNER JOIN sys.sysusers u  ON u.uid=s.principal_id 
LEFT JOIN (
SELECT schema_Name(schema_id)  as  SchemaName, COUNT(*) AS NoObjects
FROM sys.objects 
GROUP BY schema_Name(schema_id)
) AS c
ON s.name = c.SchemaName
WHERE 16384 > s.schema_id AND s.name NOT IN ('INFORMATION_SCHEMA','sys')
ORDER BY s.name;

In [ ]:
-- Schema Extended Properties
SELECT S.name AS [Schema Name], EP.name, EP.value AS [Extended property]
FROM sys.extended_properties AS EP
INNER JOIN sys.schemas AS S ON EP.major_id = S.schema_id

In [ ]:
-- Select SSIS Packages Parameters 
SELECT    fldr.name AS FolderName ,objp.[referenced_variable_name] AS [EnvironmentVariable]
        , proj.name AS ProjectName, COALESCE('Package: ' + pkg.name, 'Project') AS Scope
        , objp.parameter_name COLLATE Latin1_General_CS_AS AS ParameterName,objp.design_default_value 
        ,objp.referenced_variable_name
        ,(SELECT TOP 1 EV.value AS VariableValue FROM SSISDB.[internal].[environment_variables] EV WHERE EV.name=objp.referenced_variable_name) AS [value]
FROM SSISDB.catalog.folders AS fldr
   INNER JOIN  SSISDB.catalog.projects proj ON proj.folder_id = fldr.folder_id
   LEFT JOIN SSISDB.catalog.object_parameters objp  ON objp.project_id = proj.project_id
   LEFT JOIN SSISDB.catalog.packages pkg ON objp.object_name = pkg.name AND objp.project_id = pkg.project_id